In [1]:
import pandas as pd
path_train=r'C:\Users\acer\Desktop\skillfactory\6\hw06_data\train.csv'
train=pd.read_csv(path_train, sep='\t')

train.head(2).T

,0,1
id,0,1
name,Заведующий отделом/секцией в магазин YORK (Уру...,Наладчик станков и манипуляторов с ПУ
description,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,Обязанности:работа на токарных станках с ЧПУ T...
target,1,0


In [2]:
path_test=r'C:\Users\acer\Desktop\skillfactory\6\hw06_data\test.csv'
test=pd.read_csv(path_test, sep='\t')

test.head(2).T

,0,1
id,200000,200001
name,Дизайнер-консультант мебели,Продавец-консультант (ТЦ на Пушкина)
description,<p><strong>Обязанности:</strong></p> <ul> <li>...,<p><strong>Обязанности</strong>:</p> <p>∙ конс...


In [3]:
vacancies=train.drop(['target'], axis=1).append(test, ignore_index=True)

In [4]:
vacancies['vacancy']=vacancies['name']+' '+vacancies['description']

In [5]:
vacancies_list=list(vacancies['vacancy'])

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfCounter=TfidfVectorizer()

analyze  = tfCounter.build_analyzer()
X        = tfCounter.fit_transform(vacancies_list)

In [22]:
X_train=X[:200000, :]
X_test=X[200000:, :]

In [24]:
y_train=train['target'].values
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(200000, 244936)
(200000,)
(170179, 244936)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from datetime import datetime

param_grid_lr = {
    'penalty': ['l1', 'l2'],
    'C': [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0],
    'class_weight': [None, 'balanced']
                }
n_folds = 10
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=123)

model_lr  = LogisticRegression(random_state=123)
random_search_lr = RandomizedSearchCV(model_lr, param_distributions=param_grid_lr, n_iter=20, 
                                   cv=cv, scoring='roc_auc', random_state=123)
dt_start=datetime.now()
random_search_lr.fit(X_train, y_train)
dt_end=datetime.now()
print("Логистическая регрессия, всего: {0:.2f} минут".format(((dt_end - dt_start).total_seconds())/60))
print('---------------------------------------------------------------------------------', '\n')
print('Логистическая регрессия, лучшая модель:', random_search_lr.best_params_)
print('---------------------------------------------------------------------------------', '\n')
print('Логистическая регрессия, лучший roc_auc:', random_search_lr.best_score_)
print('-------------------------------------------------------------------------------------------------------', '\n')

Логистическая регрессия, всего: 32.54 минут
--------------------------------------------------------------------------------- 

Логистическая регрессия, лучшая модель: {'penalty': 'l1', 'class_weight': None, 'C': 1.0}
--------------------------------------------------------------------------------- 

Логистическая регрессия, лучший roc_auc: 0.9904133068891172
------------------------------------------------------------------------------------------------------- 



In [25]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(penalty='l1', class_weight=None, C=1.0, random_state=123)
model.fit(X_train, y_train)
predicted=model.predict(X_test)
y_pred_proba=model.predict_proba(X_test)

In [26]:
X_test.shape

(170179, 244936)

In [27]:
answer_proba=pd.DataFrame(y_pred_proba[:, 0], columns=['target'])
answer_proba['id']=test['id']

res_proba=answer_proba[['id', 'target']].copy()
res_proba.to_csv('data/VacancyClassificationSF05_20180724.csv', sep=',', header=True, index=False)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from datetime import datetime

tfCounter=TfidfVectorizer(ngram_range=(1,2))

analyze  = tfCounter.build_analyzer()
X        = tfCounter.fit_transform(vacancies_list)

X_train=X[:200000, :]
X_test=X[200000:, :]
y_train=train['target'].values

param_grid_lr = {
    'penalty': ['l1', 'l2'],
    'C': [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0],
    'class_weight': [None, 'balanced']
                }
n_folds = 10
cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=123)

model_lr  = LogisticRegression(random_state=123)
random_search_lr = RandomizedSearchCV(model_lr, param_distributions=param_grid_lr, n_iter=20, 
                                   cv=cv, scoring='roc_auc', random_state=123)
dt_start=datetime.now()
random_search_lr.fit(X_train, y_train)
dt_end=datetime.now()
print("Логистическая регрессия, всего: {0:.2f} минут".format(((dt_end - dt_start).total_seconds())/60))
print('---------------------------------------------------------------------------------', '\n')
print('Логистическая регрессия, лучшая модель:', random_search_lr.best_params_)
print('---------------------------------------------------------------------------------', '\n')
print('Логистическая регрессия, лучший roc_auc:', random_search_lr.best_score_)
print('-------------------------------------------------------------------------------------------------------', '\n')

Логистическая регрессия, всего: 133.92 минут
--------------------------------------------------------------------------------- 

Логистическая регрессия, лучшая модель: {'penalty': 'l2', 'class_weight': None, 'C': 1.0}
--------------------------------------------------------------------------------- 

Логистическая регрессия, лучший roc_auc: 0.9911341353382709
------------------------------------------------------------------------------------------------------- 



In [10]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(penalty='l2', class_weight=None, C=1.0, random_state=123)
model.fit(X_train, y_train)
predicted=model.predict(X_test)
y_pred_proba=model.predict_proba(X_test)

answer_proba=pd.DataFrame(y_pred_proba[:, 0], columns=['target'])
answer_proba['id']=test['id']

res_proba=answer_proba[['id', 'target']].copy()
res_proba.to_csv('data/VacancyClassificationSF05_20180725.csv', sep=',', header=True, index=False)